Quick hack to add the left and right parent scorpio designations to the recombinants CSV. Uses a mapping from the most common pango assignment to scorpio from the overall dataset.


In [1]:
import sys
sys.path.append("../../sc2ts")
import sc2ts
ds = sc2ts.Dataset("../../sc2ts/data/viridian_mafft_2024-10-14_v1.vcz.zip")
ds

In [22]:
import pandas as pd

df_recomb = pd.read_csv("../data/recombinants.csv", parse_dates=["date_added"])
df_recomb

,recombinant,descendants,sample,sample_id,sample_pango,interval_left,interval_right,num_mutations,date_added,group_id,...,Viridian_amplicon_scheme,Platform,Artic_primer_version,break_near_missing_run_12_7,net_min_supporting_loci_lft,net_min_supporting_loci_rgt,net_min_supporting_loci_lft_rgt_ge_4,parent_left_scorpio,parent_right_scorpio,sample_scorpio
0,1530,1,1529,ERR4437465,B.1.157,8783,13617,2,2020-03-22,052b938d3189a1c873abd3ffd894e4c6,...,COVID-ARTIC-V3,ILLUMINA,3,False,5,5,True,.,.,.
1,22500,1,22499,ERR4638271,B.1.1.217,26528,26714,2,2020-07-23,96ae2d6491a4e2d9432c231e6f2bb7a0,...,COVID-ARTIC-V3,ILLUMINA,3,False,11,2,False,.,.,.
2,26465,54,26464,ERR4615866,B.1.221,15325,21855,1,2020-08-24,dccf6a6ff6f85d67cdd5a7e5c0e5d758,...,COVID-ARTIC-V3,ILLUMINA,3,False,5,9,True,.,.,.
3,27003,3,27002,ERR4671078,B.1.160,22993,25563,2,2020-08-26,0c768f9bb44f8c5d4daf59582c6c9a16,...,COVID-ARTIC-V3,OXFORD_NANOPORE,3,False,13,10,True,.,.,.
4,28379,3,28378,SRR21719160,B.1.426,6542,9515,0,2020-09-02,d856a9a4583881e765e24caeb4533948,...,COVID-ARTIC-V3,ILLUMINA,.,False,4,18,True,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,1430261,1,1430260,ERR10839848,CH.1.1,19327,20741,2,2023-01-23,d5ca15babe3403d4d5764e614d8cf68f,...,COVID-ARTIC-V4.1,ILLUMINA,4.1alt,False,17,22,True,Omicron (XBB.1-like),Omicron (BA.2-like),Omicron (BA.2-like)
925,1430452,1,1430451,SRR23358540,CH.1.1,9867,12160,1,2023-01-23,b7c8e0260ee65249105de96d2a5c50e3,...,COVID-ARTIC-V4.1,ILLUMINA,.,False,12,29,True,Omicron (BA.5-like),Omicron (BA.2-like),Omicron (BA.2-like)
926,1431988,1,1431987,ERR10931828,CH.1.1,19887,21811,2,2023-01-29,efa2dbff2716ef08d00943f0d5c83395,...,COVID-ARTIC-V4.1,ILLUMINA,4.1alt,False,5,4,True,Omicron (BA.2-like),Omicron (BA.2-like),Omicron (BA.2-like)
927,1432902,2,1432901,ERR10921923,BY.1.1,20742,21767,3,2023-02-02,ef73940c5b9a228b0c1ab15502a128dd,...,COVID-ARTIC-V4.1,ILLUMINA,.,False,22,24,True,Omicron (BA.5-like),Omicron (BA.2-like),Omicron (BA.2-like)


In [7]:
df_recomb.sample_pango

0        B.1.157
1      B.1.1.217
2        B.1.221
3        B.1.160
4        B.1.426
         ...    
924       CH.1.1
925       CH.1.1
926       CH.1.1
927       BY.1.1
928    XBB.1.5.4
Name: sample_pango, Length: 929, dtype: object

In [27]:
# Build the mapping for scorpio
pango = ds["sample_Viridian_pangolin"][:]
scorpio = ds["sample_Viridian_scorpio"][:]
sample_id = ds["sample_id"]
df = pd.DataFrame({"pango": pango, "scorpio": scorpio, "sample_id": sample_id})
df

,pango,scorpio,sample_id
0,A,.,SRR11772659
1,A,.,SRR11597132
2,B.1,.,SRR12162233
3,B.1,.,SRR12162234
4,A,.,SRR11597217
...,...,...,...
4484152,.,.,ERR13177478
4484153,.,.,ERR13177479
4484154,.,.,ERR13177480
4484155,.,.,ERR13177481


In [5]:
pango2scorpio = df.groupby("pango").agg(pd.Series.mode)["scorpio"].to_dict()

In [6]:
pango2scorpio["Unknown (R)"] = "Unknown (R)"

In [24]:
cols = {"parent_left": [], "parent_right":[]}
for _, row in df_recomb.iterrows():
    for key, col in cols.items():
        pango = row[key + "_pango"]
        scorpio = pango2scorpio[pango]
        col.append(scorpio)

for k, v in cols.items():
    df_recomb[k + "_scorpio"] = v

In [25]:
df_recomb.parent_left_scorpio

0                         .
1                         .
2                         .
3                         .
4                         .
               ...         
924    Omicron (XBB.1-like)
925     Omicron (BA.5-like)
926     Omicron (BA.2-like)
927     Omicron (BA.5-like)
928    Omicron (XBB.1-like)
Name: parent_left_scorpio, Length: 929, dtype: object

In [29]:
df = df.set_index("sample_id")

In [30]:
df.loc[df_recomb.sample_id].scorpio

sample_id
ERR4437465                          .
ERR4638271                          .
ERR4615866                          .
ERR4671078                          .
SRR21719160                         .
                        ...          
ERR10839848       Omicron (BA.2-like)
SRR23358540       Omicron (BA.2-like)
ERR10931828       Omicron (BA.2-like)
ERR10921923       Omicron (BA.2-like)
ERR10933490    Omicron (XBB.1.5-like)
Name: scorpio, Length: 929, dtype: object

In [31]:
df_recomb["sample_scorpio"] = df.loc[df_recomb.sample_id].scorpio.values
df_recomb

,recombinant,descendants,sample,sample_id,sample_pango,interval_left,interval_right,num_mutations,date_added,group_id,...,Viridian_amplicon_scheme,Platform,Artic_primer_version,break_near_missing_run_12_7,net_min_supporting_loci_lft,net_min_supporting_loci_rgt,net_min_supporting_loci_lft_rgt_ge_4,parent_left_scorpio,parent_right_scorpio,sample_scorpio
0,1530,1,1529,ERR4437465,B.1.157,8783,13617,2,2020-03-22,052b938d3189a1c873abd3ffd894e4c6,...,COVID-ARTIC-V3,ILLUMINA,3,False,5,5,True,.,.,.
1,22500,1,22499,ERR4638271,B.1.1.217,26528,26714,2,2020-07-23,96ae2d6491a4e2d9432c231e6f2bb7a0,...,COVID-ARTIC-V3,ILLUMINA,3,False,11,2,False,.,.,.
2,26465,54,26464,ERR4615866,B.1.221,15325,21855,1,2020-08-24,dccf6a6ff6f85d67cdd5a7e5c0e5d758,...,COVID-ARTIC-V3,ILLUMINA,3,False,5,9,True,.,.,.
3,27003,3,27002,ERR4671078,B.1.160,22993,25563,2,2020-08-26,0c768f9bb44f8c5d4daf59582c6c9a16,...,COVID-ARTIC-V3,OXFORD_NANOPORE,3,False,13,10,True,.,.,.
4,28379,3,28378,SRR21719160,B.1.426,6542,9515,0,2020-09-02,d856a9a4583881e765e24caeb4533948,...,COVID-ARTIC-V3,ILLUMINA,.,False,4,18,True,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,1430261,1,1430260,ERR10839848,CH.1.1,19327,20741,2,2023-01-23,d5ca15babe3403d4d5764e614d8cf68f,...,COVID-ARTIC-V4.1,ILLUMINA,4.1alt,False,17,22,True,Omicron (XBB.1-like),Omicron (BA.2-like),Omicron (BA.2-like)
925,1430452,1,1430451,SRR23358540,CH.1.1,9867,12160,1,2023-01-23,b7c8e0260ee65249105de96d2a5c50e3,...,COVID-ARTIC-V4.1,ILLUMINA,.,False,12,29,True,Omicron (BA.5-like),Omicron (BA.2-like),Omicron (BA.2-like)
926,1431988,1,1431987,ERR10931828,CH.1.1,19887,21811,2,2023-01-29,efa2dbff2716ef08d00943f0d5c83395,...,COVID-ARTIC-V4.1,ILLUMINA,4.1alt,False,5,4,True,Omicron (BA.2-like),Omicron (BA.2-like),Omicron (BA.2-like)
927,1432902,2,1432901,ERR10921923,BY.1.1,20742,21767,3,2023-02-02,ef73940c5b9a228b0c1ab15502a128dd,...,COVID-ARTIC-V4.1,ILLUMINA,.,False,22,24,True,Omicron (BA.5-like),Omicron (BA.2-like),Omicron (BA.2-like)


In [32]:
df_recomb.to_csv("../data/recombinants.csv", index=False)

Add pangolin assignments for parents which are internal nodes

In [23]:
# Pangolin/scorpio assignments for internal nodes using pangolin v4.3.1 and pangolin-data v1.2.1
pango_df = pd.read_csv("../data/lineage_report.pangolin-data_v1.21.csv.gz")
pango_df.columns

Index(['taxon', 'lineage', 'conflict', 'ambiguity_score', 'scorpio_call',
       'scorpio_support', 'scorpio_conflict', 'scorpio_notes', 'version',
       'pangolin_version', 'scorpio_version', 'constellation_version',
       'is_designated', 'qc_status', 'qc_notes', 'note'],
      dtype='object')

In [24]:
parent_left_labels = []
parent_right_labels = []
num_parent_left_internal = 0
num_parent_right_internal = 0

for row in df_recomb.itertuples():
    # Left parent
    is_taxon_parent_left = pango_df.taxon == "n" + str(row.parent_left)
    if any(is_taxon_parent_left):
        assert sum(is_taxon_parent_left) == 1
        label = pango_df.loc[is_taxon_parent_left, "lineage"].values[0]
        parent_left_labels.append(label)
        num_parent_left_internal += 1
    else:
        parent_left_labels.append(row.parent_left_pango)

    # Right parent
    is_taxon_parent_right = pango_df.taxon == "n" + str(row.parent_right)
    if any(is_taxon_parent_right):
        assert sum(is_taxon_parent_right) == 1
        label = pango_df.loc[is_taxon_parent_right, "lineage"].values[0]
        parent_right_labels.append(label)
        num_parent_right_internal += 1
    else:
        parent_right_labels.append(row.parent_right_pango)

assert len(parent_left_labels) == len(df_recomb)
assert len(parent_right_labels) == len(df_recomb)

print(f"Internal left parents: {num_parent_left_internal}")
print(f"Internal right parents: {num_parent_right_internal}")

Internal left parents: 321
Internal right parents: 447


In [ ]:
df_recomb["parent_left_pango"] = parent_left_labels
df_recomb["parent_right_pango"] = parent_right_labels

In [16]:
df_recomb.to_csv("../data/recombinants.csv", index=False)